# Influx stack demonstration

In this notebook we present the main concepts of InfluxDB. The we use the influxdb Python client to get SQuaSH metrics into InflunxDB. We demonstrate the use of Chronograf for data exploration and dashboarding, and Kapacitor for creating alerting rules.


## Concepts

- InfluxDB [getting-started guide]( https://docs.influxdata.com/influxdb/v1.6/introduction/getting-started/)
- SQL to InfluxDB [terminology crosswalk](https://docs.influxdata.com/influxdb/v1.6/concepts/crosswalk/)

"Points" are discrete samples of a metric. Points are written to InfluxDB using the "Line Protocol":

```
<measurement>[,<tag-key>=<tag-value>...] <field-key>=<field-value>[,<field2-key>=<field2-value>...] [unix-nano-timestamp]
```

"Measurements" acts as a container for tags, fields, and the time column, and the measurement name is the description of the data that are stored in the associated fields. Measurement names are strings. A measurement is conceptually similar to an SQL table.

Tags are indexed while fields are not. So you should consider to turn fields into tags if you they are used to constraint your queries.

Examples: 

1. Insert a single time series point using the CLI

```
> USE mydb
> INSERT cpu,host=server01,region=us_west value=0.64
```

See also [writing and exploring data](https://docs.influxdata.com/influxdb/v1.6/introduction/getting-started/#writing-and-exploring-data).

2. Inserting a single time series point using the [HTTP API](https://docs.influxdata.com/influxdb/v1.6/guides/writing_data/)

```
$ curl -XPOST "http://localhost:8086/write?db=mydb" -d 'cpu,host=server01,region=uswest value=0.64'
```

3. Insert a single time series point using the [influxdb Python client](https://github.com/influxdata/influxdb-python)

```python
data = [
    {
        "measurement": "cpu",
        "tags": {
            "host": "server01",
            "region": "us_west"
        },
        "fields": {
            "value": 0.64
        }
    }
]
from influxdb import InfluxDBClient
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'mydb')
client.write_points(data)

```

## Running InfluxDB + Chronograf + Kapacitor

In [ ]:
%%bash
docker-compose up -d

On the terminal you can open the InfluxDB CLI with

```bash
$ docker-compose run influxdb-cli
Starting influx-demo_influxdb_1 ... done
Connected to http://influxdb:8086 version 1.6.3
InfluxDB shell version: 1.6.3
> show databases
name: databases
name
----
telegraf
_internal
chronograf
squash
```

Cronograf will run at http://localhost:8888

## Getting SQuaSH metrics into InfluxDB

Approach #1: each metric in SQuaSH is an InfluxDB measurement

In [1]:
from influxdb import InfluxDBClient

client = InfluxDBClient('localhost', 8086, 'root', 'root', 'squash')
client.create_database('squash')

In [ ]:
import requests

SQUASH_API_URL = "https://squash-restful-api-demo.lsst.codes/"
jobs = requests.get(SQUASH_API_URL + "/jobs").json()

In [ ]:
from datetime import datetime

for job_id in jobs['ids']:

    r = requests.get(SQUASH_API_URL + "/job/{}".format(job_id)).json()
    
    if r['ci_dataset'] == 'unknown' or r['ci_dataset'] == 'decam':
        continue
    
    print('Sending point for job {}...'.format(job_id))

    points = []
    
    for meas in r['measurements']:
        
        point = dict()
        
        point['measurement'] = meas['metric']
       
        point['tags'] =  {'filter_name': r['meta']['filter_name'], 
                          'dataset':  r['ci_dataset']}
                          
        point['time'] = r['date_created']
        
        point['fields'] = {'value': meas['value']}
        
        points.append(point)
        
    
    client.write_points(points)

Approach #2: each verification package in SQuaSH is an InfluxDB measurement

In [9]:
from influxdb import InfluxDBClient

client = InfluxDBClient('localhost', 8086, 'root', 'root', 'squash_2')
client.create_database('squash_2')

In [10]:
import requests

SQUASH_API_URL = "https://squash-restful-api-demo.lsst.codes/"
jobs = requests.get(SQUASH_API_URL + "/jobs").json()

In [11]:
from datetime import datetime

for job_id in jobs['ids']:

    r = requests.get(SQUASH_API_URL + "/job/{}".format(job_id)).json()
    
    if r['ci_dataset'] == 'unknown' or r['ci_dataset'] == 'decam':
        continue
    
    print('Sending point for job {}...'.format(job_id))

    points = []
    
    for meas in r['measurements']:
        
        point = dict()
        
        point['measurement'] = meas['metric'].split('.')[0]
       
        point['tags'] =  {'filter_name': r['meta']['filter_name'], 
                          'dataset':  r['ci_dataset']}
                          
        point['time'] = r['date_created']
        
        point['fields'] = {meas['metric']: meas['value']}
        
        points.append(point)
        
    
    client.write_points(points)

Sending point for job 1...
Sending point for job 3...
Sending point for job 5...
Sending point for job 7...
Sending point for job 9...
Sending point for job 11...
Sending point for job 13...
Sending point for job 14...
Sending point for job 15...
Sending point for job 16...
Sending point for job 17...
Sending point for job 18...
Sending point for job 19...
Sending point for job 20...
Sending point for job 21...
Sending point for job 22...
Sending point for job 23...
Sending point for job 24...
Sending point for job 25...
Sending point for job 26...
Sending point for job 27...
Sending point for job 28...
Sending point for job 29...
Sending point for job 30...
Sending point for job 31...
Sending point for job 32...
Sending point for job 33...
Sending point for job 34...
Sending point for job 35...
Sending point for job 36...
Sending point for job 37...
Sending point for job 38...
Sending point for job 39...
Sending point for job 40...
Sending point for job 41...
Sending point for job 42.

Sending point for job 293...
Sending point for job 294...
Sending point for job 295...
Sending point for job 296...
Sending point for job 297...
Sending point for job 298...
Sending point for job 299...
Sending point for job 300...
Sending point for job 301...
Sending point for job 302...
Sending point for job 303...
Sending point for job 304...
Sending point for job 305...
Sending point for job 306...
Sending point for job 307...
Sending point for job 308...
Sending point for job 309...
Sending point for job 310...
Sending point for job 311...
Sending point for job 312...
Sending point for job 313...
Sending point for job 314...
Sending point for job 315...
Sending point for job 316...
Sending point for job 317...
Sending point for job 318...
Sending point for job 319...
Sending point for job 320...
Sending point for job 321...
Sending point for job 322...
Sending point for job 323...
Sending point for job 324...
Sending point for job 325...
Sending point for job 326...
Sending point 

Sending point for job 576...
Sending point for job 577...
Sending point for job 578...
Sending point for job 579...
Sending point for job 580...
Sending point for job 581...
Sending point for job 582...
Sending point for job 583...
Sending point for job 584...
Sending point for job 585...
Sending point for job 586...
Sending point for job 587...
Sending point for job 588...
Sending point for job 589...
Sending point for job 590...
Sending point for job 591...
Sending point for job 592...
Sending point for job 593...
Sending point for job 594...
Sending point for job 595...
Sending point for job 596...
Sending point for job 597...
Sending point for job 598...
Sending point for job 599...
Sending point for job 600...
Sending point for job 601...
Sending point for job 602...
Sending point for job 603...
Sending point for job 604...
Sending point for job 605...
Sending point for job 606...
Sending point for job 607...
Sending point for job 608...
Sending point for job 609...
Sending point 

Sending point for job 954...
Sending point for job 955...
Sending point for job 956...
Sending point for job 957...
Sending point for job 958...
Sending point for job 959...
Sending point for job 960...
Sending point for job 961...
Sending point for job 962...
Sending point for job 963...
Sending point for job 964...
Sending point for job 965...
Sending point for job 966...
Sending point for job 967...
Sending point for job 968...
Sending point for job 969...
Sending point for job 970...
Sending point for job 971...
Sending point for job 972...
Sending point for job 973...
Sending point for job 974...
Sending point for job 975...
Sending point for job 976...
Sending point for job 977...
Sending point for job 978...
Sending point for job 979...
Sending point for job 980...
Sending point for job 981...
Sending point for job 982...
Sending point for job 983...
Sending point for job 984...
Sending point for job 985...
Sending point for job 986...
Sending point for job 987...
Sending point 

Sending point for job 1235...
Sending point for job 1236...
Sending point for job 1237...
Sending point for job 1238...
Sending point for job 1239...
Sending point for job 1240...
Sending point for job 1241...
Sending point for job 1242...
Sending point for job 1243...
Sending point for job 1244...
Sending point for job 1245...
Sending point for job 1246...
Sending point for job 1247...
Sending point for job 1248...
Sending point for job 1249...
Sending point for job 1250...
Sending point for job 1251...
Sending point for job 1252...
Sending point for job 1253...
Sending point for job 1254...
Sending point for job 1255...
Sending point for job 1256...
Sending point for job 1257...
Sending point for job 1258...
Sending point for job 1259...
Sending point for job 1260...
Sending point for job 1261...
Sending point for job 1262...
Sending point for job 1263...
Sending point for job 1264...
Sending point for job 1265...
Sending point for job 1266...
Sending point for job 1267...
Sending po

Sending point for job 1524...
Sending point for job 1525...
Sending point for job 1526...
Sending point for job 1527...
Sending point for job 1528...
Sending point for job 1529...
Sending point for job 1530...
Sending point for job 1531...
Sending point for job 1532...
Sending point for job 1533...
Sending point for job 1534...
Sending point for job 1535...
Sending point for job 1536...
Sending point for job 1537...
Sending point for job 1538...
Sending point for job 1539...
Sending point for job 1540...
Sending point for job 1541...
Sending point for job 1542...
Sending point for job 1543...
Sending point for job 1544...
Sending point for job 1545...
Sending point for job 1546...
Sending point for job 1547...
Sending point for job 1548...
Sending point for job 1549...
Sending point for job 1550...
Sending point for job 1551...
Sending point for job 1552...
Sending point for job 1553...
Sending point for job 1554...
Sending point for job 1555...
Sending point for job 1556...
Sending po

## Alerting

Alerting is done with [Kapacitor](https://docs.influxdata.com/kapacitor) which has a nice integration with Chronograf, not limited to alerting though.

## Discussion

### InfluxDB

- Built-in HTTP API (but not RESTful)
- CLI
- InfluxDB Python client
- SQL-like query language
- Functions like mean(), median(), min() and max()


### Chronograf

- Written in Go and React.js
- Explore mode, query builder is great, multiple queries in a single graph
- Query builder, ability to edit and test the actual query before submission
- Easy to create dashboards, nice control of the graph properties, presentation mode
- [Template variables are great to customize dashboards](https://docs.influxdata.com/chronograf/v1.6/guides/dashboard-template-variables/)
- [You can export your dashboard definition to JSON](https://www.influxdata.com/blog/chronograf-dashboard-definitions/)
- Easy to configure alerting rules
- You can export data from the system (download CSV from the UI, from the [HTTP API](http://localhost:8888/docs#tag/measurements) or from thr CLI)
- [Combine metrics and logs dashboard](https://docs.influxdata.com/chronograf/v1.6/guides/analyzing-logs/#logs-in-dashboards)

## Kapacitor

- Alert rules
- Time series processing 
- [HTTP API](https://docs.influxdata.com/kapacitor/v1.5/working/api/)